### Determine Optimal Portfolio Allocation Using Monte Carlo

In [14]:
import datetime as dt
import yfinance as yf
import numpy as np

# 10 Companies to Design Portfolio Around
tickers = ["coke", 'googl', 'hd', 'ford', 'smg', 'jnj', 'ag']
start = "2000-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

[*********************100%***********************]  7 of 7 completed


In [21]:
# Set Date as Index
df_stock_data = df_portfolio.set_index("Date").copy(deep=True)
df_stock_data = df_stock_data["Adj Close"].dropna()

companies = df_stock_data.columns

for company in companies:
    df_stock_data[f"{company} return"] = np.log(df_stock_data[company].pct_change())

df_stock_data.dropna(inplace=True)

df_return_series = df_stock_data.loc[6:]
df_return_series

c:\Users\PCSale123\OneDrive\Desktop\Data Science\Finance\financial-analysis\venv\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\PCSale123\OneDrive\Desktop\Data Science\Finance\financial-analysis\venv\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\PCSale123\OneDrive\Desktop\Data Science\Finance\financial-analysis\venv\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\PCSale123\OneDrive\Desktop\Data Science\Finance\financial-analysis\venv\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\PCSale123\OneDrive\Desktop\Data Science\Finance\financial-analysi

TypeError: cannot do slice indexing on DatetimeIndex with these indexers [6] of type int